In [1]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
import os

In [5]:
base_dir = './train/'

In [6]:
base_dir

'./train/'

In [7]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir, 
    target_size = (IMAGE_SIZE, IMAGE_SIZE), 
    batch_size=BATCH_SIZE,
    subset = 'training'
)
val_generator = datagen.flow_from_directory(
base_dir,
target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 14388 images belonging to 74 classes.
Found 3566 images belonging to 74 classes.


In [8]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'Afghan': 0, 'African_wild_dog': 1, 'Airedale': 2, 'American Hairless': 3, 'American Spaniel': 4, 'Basenji': 5, 'Basset': 6, 'Beagle': 7, 'Bearded collie': 8, 'Bermaise': 9, 'Bichon Frise': 10, 'Blenheim': 11, 'Bloodhound': 12, 'Bluetick': 13, 'Border collie': 14, 'Borzoi': 15, 'Boston Terrier': 16, 'Boxer': 17, 'Bull Mastiff': 18, 'Bull Terrier': 19, 'Bulldog': 20, 'Cairn': 21, 'Chihuahua': 22, 'Chinese Crested': 23, 'Chow': 24, 'Clumber': 25, 'Cockapoo': 26, 'Cocker': 27, 'Collie': 28, 'Corgi': 29, 'Coyote': 30, 'Dalmation': 31, 'Dhole': 32, 'Dingo': 33, 'Doberman': 34, 'Elk hound': 35, 'French bulldog': 36, 'German Sheperd': 37, 'Golden Retriever': 38, 'Great Dane': 39, 'Great Perenees': 40, 'Greyhound': 41, 'Groenendael': 42, 'Himalayan Sheepdog': 43, 'Human Beings': 44, 'Irish Spaniel': 45, 'Irish Wolfhound': 46, 'Japanese Spaniel': 47, 'Japanese Spitz': 48, 'Komondor': 49, 'Labradoodle': 50, 'Labrador': 51, 'Lhasa': 52, 'Malinois': 53, 'Maltese': 54, 'Mexican Hairless': 55, 'New

In [9]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape = IMG_SHAPE,
    include_top = False,
    weights = 'imagenet'
)

In [10]:
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3,activation='relu'),
    tf.keras.layers.Dropout(0, 2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(74,activation='softmax')
])

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
epochs = 35
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/35
225/225 [==============================] - 952s 4s/step - loss: 1.6346 - accuracy: 0.5963 - val_loss: 0.6812 - val_accuracy: 0.8197
Epoch 2/35
225/225 [==============================] - 667s 3s/step - loss: 0.6270 - accuracy: 0.8189 - val_loss: 0.5837 - val_accuracy: 0.8396
Epoch 3/35
225/225 [==============================] - 661s 3s/step - loss: 0.4401 - accuracy: 0.8579 - val_loss: 0.5753 - val_accuracy: 0.8421
Epoch 4/35
225/225 [==============================] - 2464s 11s/step - loss: 0.3243 - accuracy: 0.8910 - val_loss: 0.5523 - val_accuracy: 0.8559
Epoch 5/35
225/225 [==============================] - 309s 1s/step - loss: 0.2552 - accuracy: 0.9103 - val_loss: 0.5496 - val_accuracy: 0.8702
Epoch 6/35
225/225 [==============================] - 312s 1s/step - loss: 0.2028 - accuracy: 0.9265 - val_loss: 0.5307 - val_accuracy: 0.8718
Epoch 7/35
225/225 [==============================] - 317s 1s/step - loss: 0.1536 - accuracy: 0.9440 - val_loss: 0.5483 - val_accuracy: 0.86

In [13]:
keras_file = 'predict.h5'

In [14]:
keras.models.save_model(model, keras_file)

In [15]:
model = keras.models.load_model(keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [16]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\its\AppData\Local\Temp\tmpr8z4ou7w\assets


INFO:tensorflow:Assets written to: C:\Users\its\AppData\Local\Temp\tmpr8z4ou7w\assets


In [17]:
open('model.tflite','wb').write(tflite_model)

10351048